In [ ]:
from itertools import product
positive = ["嗯嗯是的呢", "是的，我问的就是这个问题", "没错", "是的呢", "是的", "嗯嗯", "是这个问题", "没错，就是这个问题", "你理解的没错", "没有问题", "你说的没错，就是这个问题", "对的", "没错，帮我找一下这个问题的答案吧", "确定"]
negative = ["不对", "不对吧", "不是", "不是吧","错的吧","呃不对吧","呃不是"]

In [2]:
# 统计已标注的模板数和总共的模板数
import json
for intent in ["Recommend", "Request", "Confirm", "Doubt", "Browse", "Download"]:
    with open("%s_user.json" % intent, "r", encoding="utf-8") as fw:
        data=json.load(fw)
    d=[i for i in data if len(i["message"])>0]
    print("%s_user.json" % intent, len(d), len(data))
    with open("%s_system.json" % intent, "r", encoding="utf-8") as fw:
        data=json.load(fw)
    d=[i for i in data if len(i["message"])>0]
    print("%s_system.json" % intent, len(d), len(data))
# with open("z.json", "r", encoding="utf-8") as fw:
#     data=json.load(fw)
# d=[i for i in data if len(i["message"])>0]
# print(len(d), len(data))

Recommend_user.json 300 300
Recommend_system.json 221 221
Request_user.json 48 48
Request_system.json 625 625
Confirm_user.json 22 22
Confirm_system.json 23 23
Doubt_user.json 16 16
Doubt_system.json 17 17
Browse_user.json 6 6
Browse_system.json 6 6
Download_user.json 5 5
Download_system.json 3 3


In [33]:

# 系统回复具体信息的语言模板
dict={
    "Conference": {
        "fullName": "的全称是XXX",
        "paperDue": "的截稿时间是XXX", 
        "noticeDue": "的通知时间是XXX",
        "heldDate": "的举行时间是XXX", 
        "abstractDue": "的摘要截稿时间是XXX",
        "finalDue": "的最终版截稿时间是XXX",
        "location": "的举行地点是XXX",
        "category": "的研究领域有XXX",
        "ccf": "是XXX会议",
        "website": "的官网是XXX",
        "bestPaper": "的最佳论文有XXX",
        "confRelated": "的相关会议有XXX",
        "journalRelated": "的相关期刊有XXX"
    },
    "Paper": {
        "doi": "的doi是XXX",
        "author": "的作者有XXX",
        "venue": "是在XXX上发表的",
        "year": "是在XXX发表的",
        "institution": "的发表机构是XXX",
        "abstract": "的摘要是XXX",
        "pdf": "的pdf网址是XXX",
        "video": "的讲解视频网址有XXX",
        "blog": "的讲解博客网址有XXX"
    },
    "Journal": {
        "ISSN": "的ISSN是XXX",
        "E_ISSN": "的E_ISSN是XXX",
        "IF": "的IF影响因子是XXX", 
        "h_index": "的h5指数是XXX",
        "self_citing": "的自引率是XXX",
        "ccf": "是XXX期刊", 
        "sci": "是XXX期刊",
        "website": "的官网是XXX",
        "submit": "的投稿网址是XXX",
        "OA": "XXX开放访问",
        "category": "的研究领域有XXX",
        "speed": "的审稿速度是XXX",
        "rate": "的录用比例是XXX",
    },
    "Author": {
        "field": "的研究领域有XXX",
        "affiliation": "的工作机构是XXX",
        "position": "的职称是XXX",
        "citations": "的总引用量是XXX",
        "h_index": "的h5指数是XXX",
        "pubs": "的总发表数是XXX"
    },
    "Institution": {
        "address": "的地址是XXX",
        "website": "的网址是XXX"
    }
}
# 槽值的中文意思
trans = {
    "fullName": "全称",
    "paperDue": "截稿时间", 
    "noticeDue": "通知时间",
    "heldDate": "举行时间", 
    "abstractDue": "摘要截稿时间",
    "finalDue": "最终版截稿时间",
    "location": "举行地点",
    "category": "研究领域",
    "ccf": "ccf级别",
    "website": "官网",
    "bestPaper": "最佳论文",
    "confRelated": "相关会议",
    "journalRelated": "相关期刊",
    "doi": "doi",
    "author": "作者",
    "venue": "来源",
    "year": "年份",
    "institution": "发表机构",
    "abstract": "摘要",
    "pdf": "pdf网址",
    "video": "讲解视频网址",
    "blog": "讲解博客网址",
    "ISSN": "ISSN",
    "E_ISSN": "E_ISSN",
    "IF": "IF影响因子", 
    "h_index": "h5指数",
    "self_citing": "自引率",
    "sci": "sci分区",
    "submit": "投稿网址",
    "OA": "开放访问权限",
    "speed": "审稿速度",
    "rate": "录用比例",
    "field": "研究领域",
    "affiliation": "工作机构",
    "position": "职称",
    "citations": "总引用量",
    "pubs": "总发表数",
    "address": "地址"
}
# 默认的identify值
iden = {
    "Conference": "ACL会议",
    "Paper": "这篇论文",
    "Journal": "TOIS期刊",
    "Author": "黄河燕",
    "Institution": "北京大学",
}

In [37]:
# 自动标注Request_system.json
import json
from itertools import product
with open("Request_system.json", "r", encoding="utf-8") as fw:
    data=json.load(fw)
for i in data:
    domain,intent = i["action"][0].split("-")
    ide = "ACL2022" if domain=="Conference" and "year" in i["required_slot"] else iden[domain]
    slot = i["required_slot"][2:] if domain=="Conference" and "year" in i["required_slot"] else i["required_slot"][1:]
    if intent == "Inform":  # 根据上面的dict来标注语句
        if len(slot) == 1:
            i["message"] = [ide + dict[domain][slot[0]]]
        else: # 多个slot
            i["message"] = [ide + dict[domain][slot[0]] + "，" + dict[domain][slot[1]].replace("的","")]
    elif intent == "NoInform":  # 没有找到
        i["message"] = [
            "目前数据库中没有找到%s的%s" % (ide, '和'.join([trans[i] for i in slot])),
            "目前数据库中没有找到%s的%s，之后我们会不断完善数据，感谢您的理解" % (ide, '和'.join([trans[i] for i in slot]))
        ]
    else:  # Recommend more
        i["message"] = [
            "你想了解一下%s的%s嘛" % (ide, trans[i["requested_slot"][0]])
        ]

with open("Request_system.json", "w", encoding="utf-8") as fw:
    json.dump(data,fw,indent=4,ensure_ascii=False)

In [ ]:
# 在Request_system.json中将NoInform和Recommend合并
from itertools import product
import json
with open("Request_system.json", "r", encoding="utf-8") as fw:
    data=json.load(fw)
noinform = [i for i in data if "NoInform" in i["action"][0]]
recommend = [i for i in data if "Recommend" in i["action"][0]]
temp = product(noinform, recommend)
for i,j in temp:
    if i["action"][0].split("-")[0] != j["action"][0].split("-")[0]: #domain不同:
        continue
    if i["required_slot"][-1] == j["requested_slot"][0]: # 槽值相同
        continue
    nnk = {
        "action": i["action"]+j["action"],
        "required_slot": i["required_slot"],
        "requested_slot": j["requested_slot"],
        "description": i["description"]+"，"+j["description"],
        "message": [i["message"][0]+"，"+j["message"][0]]
    }
    data.append(nnk)
with open("Request_system.json", "w", encoding="utf-8") as fw:
    json.dump(data,fw,indent=4,ensure_ascii=False)

In [35]:
# 自动标注Confirm_system.json
positive = ["嗯嗯是的呢", "是的", "没错", "是的呢", "嗯嗯", "你理解的没错", "没有问题", "你说的没错", "对的"]
negative = ["不对，", "不对的，", "不是，", "不是的，"]
import json
from itertools import product
with open("Confirm_system.json", "r", encoding="utf-8") as fw:
    data=json.load(fw)
for i in data:
    if i["action"][0] == "General-Negative":
        domain,intent = i["action"][1].split("-")
        ide = "ACL2022" if domain=="Conference" and "year" in i["required_slot"] else iden[domain]
        slot = i["required_slot"][2:] if domain=="Conference" and "year" in i["required_slot"] else i["required_slot"][1:]
        if intent == "Inform":  # 只需要标注negative+inform情况
            if len(slot) == 1:
                i["message"] = [neg + ide + dict[domain][slot[0]] for neg in negative]
            else:
                i["message"] = [neg + ide + dict[domain][slot[0]] + "，" + dict[domain][slot[1]].replace("的","") for neg in negative]

with open("Confirm_system.json", "w", encoding="utf-8") as fw:
    json.dump(data,fw,indent=4,ensure_ascii=False)

In [45]:
# 自动标注Doubt_system.json
negative=["你说的不对","你质疑的不对","你理解不错","你记错了","不是这样的","你说的数据是错的"]
with open("Doubt_system.json", "r", encoding="utf-8") as fw:
    data=json.load(fw)
for i in data:
    if len(i["action"])>1 and i["action"][0] == "General-Negative":
        domain,intent = i["action"][1].split("-")
        ide = "ACL2022" if domain=="Conference" and "year" in i["required_slot"] else iden[domain]
        slot = i["required_slot"][2:] if domain=="Conference" and "year" in i["required_slot"] else i["required_slot"][1:]
        if intent == "Inform":  # 只需要标注negative+inform情况
            if len(slot) == 1:
                i["message"] = [neg + '，' + ide + dict[domain][slot[0]] for neg in negative]
            else:
                i["message"] = [neg + '，' + ide + dict[domain][slot[0]] + "，" + dict[domain][slot[1]].replace("的","") for neg in negative]

with open("Doubt_system.json", "w", encoding="utf-8") as fw:
    json.dump(data,fw,indent=4,ensure_ascii=False)

In [ ]:
# 生成user_template和system_template
import json
user={}
system={}
for intent in ["Recommend", "Request", "Confirm", "Doubt", "Browse", "Download"]:
    with open("%s_user.json" % intent, "r", encoding="utf-8") as fw:
        data = json.load(fw)
    for template in data:
        key = " ".join(template["action"]) + " " + " ".join(template["required_slot"]) + " " + " ".join(template["requested_slot"])
        user[key] = template
    with open("%s_system.json" % intent, "r", encoding="utf-8") as fw:
        data = json.load(fw)
    for template in data:
        key = " ".join(template["action"]) + " " + " ".join(template["required_slot"]) + " " + " ".join(template["requested_slot"])
        if key in system.keys():
            system[key]["message"].extend(template["message"])
            print(key, intent)
        else:
            system[key] = template
print(len(user), len(system))
with open("user_template.json", "w", encoding="utf-8") as fw:
    json.dump(user,fw,indent=4,ensure_ascii=False)
with open("system_template.json", "w", encoding="utf-8") as fw:
    json.dump(system,fw,indent=4,ensure_ascii=False)